In [ ]:
#!/usr/bin/env python3
'''
Main script
'''

import os
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

from CrySPY.interface import select_code
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.IO import read_input as rin
from CrySPY.start import cryspy_init, cryspy_restart

from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version

from aiida.engine import run_get_node, run, submit
from aiida.plugins import DataFactory, WorkflowFactory
import numpy as np
from aiida.orm import Code
from aiida.orm import Str, Dict, Int, List, Float
from aiida.engine import calcfunction, WorkChain, append_
from aiida.plugins import DataFactory
from itertools import cycle
import os
import io
from configparser import ConfigParser

from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal

In [ ]:
import aiida
aiida.load_profile()

In [ ]:
# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')
LammpsPotential = DataFactory('lammps.potential')
TrajectoryData = DataFactory('array.trajectory')


In [ ]:
SIMULATOR_PREFIX = 'simulator_'
ID_PREFIX = 'ID_'

In [ ]:
# nodebank settings

from tools.nodebank import NodeBank
pwd = os.getcwd()
print(pwd)
nodebank = NodeBank(pwd)


In [ ]:
pwd = os.getcwd()
cryspy_in = SinglefileData(os.path.join(pwd, 'cryspy.in0'))
inputs = {'cryspy_in': cryspy_in}
initialize_WorkChain = WorkflowFactory('cryspy.initial_structures')
result = run(initialize_WorkChain, **inputs)

In [ ]:
# workflow結果表示
result

In [ ]:
import shutil
# cryspy.inは実行中に書き換えられる。
shutil.copy("cryspy.in0", "cryspy.in")

In [ ]:
# workflowのoutputはAiiDAで扱える型。
# CrySPYの init_struc_dataへの変換を行う。

def convert_struc_dict_from_aiidatype(init_struc):
    """
    workflowのoutputはAiiDAで扱える型なので、
    CrySPYの init_struc_dataへの変換を行う。
    """
    if isinstance(init_struc, Dict):
        init_struc = init_struc.get_dict() 
    init_struc_data = {}
    for key,value in init_struc.items():
        struc = Structure.from_dict(value)
        key = int(key.replace(ID_PREFIX,""))
        init_struc_data[key] = struc #.as_dict()
        
    return init_struc_data

In [ ]:
init_struc_data = convert_struc_dict_from_aiidatype(result["init_struc"])
init_struc_data

In [ ]:
!verdi graph generate 14997

In [ ]:
@calcfunction
def aiida_load_stat():
    """
    configparserの型をAiiDAで扱えるDictに変換
    """
    stat, init_struc_data = cryspy_restart.restart()
    stat_dict = stat._sections
    return Dict(dict=stat_dict)

if False:
    @calcfunction
    def aiida_load_init_struc_data():
        stat, struc_dict = cryspy_restart.restart()
        structures = {}
        for _i, value in struc_dict.items():
            content = value.as_dict()
            key = f'ID_{_i}'
            structures[key] = content
        return Dict(dict=structures)                
    self.out('init_struc', aiida_load_init_struc_data())

stat_dic = aiida_load_stat()
stat = ConfigParser()
stat.read_dict(dictionary=stat_dic.get_dict())
# stat ConfigParser is given.


In [ ]:
os.makedirs("work", exist_ok=True)
jobs = Ctrl_job(stat, init_struc_data)
jobs.check_job()


In [ ]:
def ctrl_next_struc_step1(jobs):
    # ---------- RS
    if rin.algo == 'RS':
        next_struc_data = jobs.init_struc_data[jobs.current_id]
    # ---------- BO
    elif rin.algo == 'BO':
        next_struc_data = jobs.init_struc_data[jobs.current_id]
    # ---------- LAQA
    elif rin.algo == 'LAQA':
        if jobs.laqa_struc[jobs.current_id]:    # vacant list?
            next_struc_data = jobs.laqa_struc[jobs.current_id][-1]
        else:
            next_struc_data = jobs.init_struc_data[jobs.current_id]
    # ---------- EA
    elif rin.algo == 'EA':
        next_struc_data = jobs.init_struc_data[jobs.current_id]
    # ---------- algo is wrong
    else:
        raise ValueError('Error, algo')
        
    return next_struc_data

def ctrl_next_struc_step2(jobs, next_struc_data ):
    # ---------- common part
    # ------ in case there is no initial strucure data
    if next_struc_data is None:
        print('ID {:>6}: initial structure is None'.format(
        jobs.current_id))
        jobs.ctrl_skip()
    # ------ normal initial structure data
    else:
        if False:
            # -- prepare input files for structure optimization
            if rin.kpt_flag:
                print("kpt_flag not supported")
                raise ValueError("kpt_flag not supported")
                jobs.kpt_data = select_code.next_struc(next_struc_data,
                                                   jobs.current_id,
                                                   jobs.work_path,
                                                   jobs.kpt_data)
            else:
                print("select_code.next_struc")
                print(jobs.current_id, jobs.work_path)
                select_code.next_struc(next_struc_data, jobs.current_id,
                                   jobs.work_path)
            
        # -- prepare jobfile
        print("prepare_jobfile")
        # jobs.prepare_jobfile()
        # -- submit
        print("submit_next_struc")
        jobs.submit_next_struc(dry_run=True)
        print('ID {:>6}: submit job, Stage 1'.format(jobs.current_id))
        # -- update status
        # jobs.update_status(operation='submit')

def ctrl_next_struc(jobs):
    next_struc_data = ctrl_next_struc_step1(jobs)

    ctrl_next_struc_step2(jobs,next_struc_data)
    print("must call jobs.update_status(operation='submit')")
    
    
def handle_job_return_dict(jobs):
    print('\n# ---------- job status')
    cwd_dict = {}
    struc_dict = {}
    for cid in jobs.tmp_running:
        # ---------- set work_path and current_id
        jobs.work_path = './work/{:06}/'.format(cid)
        jobs.current_id = cid
        # ---------- handle job
        if jobs.job_stat[cid] == 'submitted':
            print('ID {:>6}: still queueing or running'.format(cid))
        elif jobs.job_stat[cid] == 'done':
            jobs.ctrl_done()
        elif jobs.job_stat[cid] == 'skip':
            jobs.ctrl_skip()
        elif jobs.job_stat[cid] == 'else':
            raise ValueError('Wrong job_stat in {}. '.format(
                jobs.work_path))
        elif jobs.job_stat[cid] == 'no_file':
            ctrl_next_struc(jobs)
            key = f'ID_{cid}'
            cwd_dict[key] = os.path.abspath(jobs.work_path)
            struc_dict[key] = jobs.init_struc_data[jobs.current_id].as_dict()
        else:
            raise ValueError('Unexpected error in {}stat_job'.format(
                jobs.work_path))
        
    return cwd_dict, struc_dict

def finish_job(jobs):
    for cid in jobs.tmp_running:
        jobs.current_id = cid
        jobs.update_status(operation='submit')
    

In [ ]:
cwd_dict, struc_dict = handle_job_return_dict(jobs)


In [ ]:
jobs.id_queueing, jobs.id_running


In [ ]:
computer_local = 'localhost'
code_lammps_opt = get_or_create_code('lammps.optimize',
                                     computer_local, 'lammps')
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 4}
}
code_lammps_opt.label

In [ ]:
# lammps potentials
pair_style = 'tersoff'
potential_dict = {
    'Ga Ga Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 1.0 1.44970 410.132 2.87 0.15 1.60916 535.199',
    'N  N  N': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 1.0 2.38426 423.769 2.20 0.20 3.55779 1044.77',
    'Ga Ga N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga N ': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 0.0 0.00000 0.00000 2.20 0.20 0.00000 0.00000',
    'N  N  Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 0.0 0.00000 0.00000 2.87 0.15 0.00000 0.00000'}
potential = DataFactory("lammps.potential")(
    type=pair_style, data=potential_dict
)
potential.attributes

In [ ]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': [],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 1000,
        'max_evaluations': 10000
        }
})


In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)

In [ ]:
jobs.id_queueing,jobs.id_running


In [ ]:
print(not (jobs.id_queueing or jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    jobs.next_sg()

# next run

In [ ]:
stat, struc_dict = cryspy_restart.restart()
jobs = Ctrl_job(stat, init_struc_data)
jobs.check_job()
cwd_dict, struc_dict = handle_job_return_dict(jobs)


In [ ]:
jobs.id_queueing, jobs.id_running


In [ ]:
cwd_dict, struc_dict


In [ ]:
len(struc_dict)


In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)
    

In [ ]:
jobs.id_queueing, jobs.id_running

In [ ]:
def convert_struc_dict_to_aiidatype(init_struc):
    """
    CrySPYの init_struc_dataを
    workflowのoutputはAiiDAで扱える型への変換を行う。
    """
    init_struc_data = {}
    for ID, value in init_struc.items():
        if isinstance(value, dict) or isinstance(value, float) or isinstance(value, int):
            struc = value
        elif isinstance(value,Structure):
            struc = value.as_dict()
        else:
            raise TypeError(f'unknown value type {type(value)}')
        key = f'{ID_PREFIX}{ID}'
        init_struc_data[key] = struc
    return init_struc_data

def convert_df_to_aiida_type(rslt_data):
    result = {}
    for label in rslt_data.columns:
        value = rslt_data[label].values.tolist()
        result[label] = value
    return result

def convert_df_from_aiida_type(rslt_data):
    if isinstance(rslt_data,Dict):
        rslt_data = rslt_data.get_dict()
    return pd.DataFrame(rslt_data)


In [ ]:
from CrySPY.IO import read_input as rin
from CrySPY.EA import ea_next_gen
import os

'''
Generational change in evolutionary algorithm
'''

import pandas as pd

from CrySPY.EA.ea_child import child_gen
from CrySPY.gen_struc.EA.select_parents import Select_parents
from CrySPY.IO import out_results
from CrySPY.IO import change_input, io_stat, pkl_data
from CrySPY.IO import read_input as rin


def next_gen_dict(stat, init_struc_data, opt_struc_data, rslt_data, ea_id_data):
    _stat = stat
    stat = ConfigParser()
    stat.read_dict(_stat)

    _init_struc_data = init_struc_data
    init_struc_data = convert_struc_dict_from_aiidatype(init_struc_data)
    _opt_struc_data = opt_struc_data
    opt_struc_data = convert_struc_dict_from_aiidatype(opt_struc_data)
    _rslt_data = rslt_data
    rslt_data = convert_df_from_aiida_type(rslt_data)
    
    # ---------- ea_id_data
    gen, id_queueing, id_running = ea_id_data["gen"], ea_id_data["id_queueing"], ea_id_data["id_running"]

    # ---------- out and log
    with open('cryspy.out', 'a') as fout:
        fout.write('# ---------- Evolutionary algorithm\n\n')
        fout.write('# ------ Generation {}\n'.format(gen + 1))
    print('# ---------- Evolutionary algorithm\n')
    print('# ------ Generation {}'.format(gen + 1))

    # ---------- current generation
    c_rslt = rslt_data[rslt_data['Gen'] == gen]
    c_fitness = c_rslt['E_eV_atom'].to_dict()    # {ID: energy, ...}

    # ---------- load ea_data, ea_data is used only in this module
    elite_struc, elite_fitness, ea_info, ea_origin = pkl_data.load_ea_data()

    # ---------- instantiate Seclect_parents class
    print('# -- select parents')
    sp = Select_parents(opt_struc_data, c_fitness, elite_struc, elite_fitness,
                        rin.fit_reverse, rin.n_fittest,
                        rin.emax_ea, rin.emin_ea)
    if rin.slct_func == 'TNM':
        sp.set_tournament(t_size=rin.t_size)
    else:
        sp.set_roulette(a=rin.a_rlt, b=rin.b_rlt)

    # ---------- generate offspring by EA
    print('# -- Generate structures')
    _, eagen = child_gen(sp, init_struc_data)

    # ---------- select elite
    if rin.n_elite > 0:
        print('# -- select elites')
        # ------ init
        all_fitness = rslt_data['E_eV_atom'].to_dict()    # {ID: energy, ..,}
        elite_struc = {}
        elite_fitness = {}
        # ------ Select_parents class also works for selecting elite structures
        se = Select_parents(opt_struc_data, all_fitness, None, None,
                            rin.fit_reverse, rin.n_elite,
                            rin.emax_ea, rin.emin_ea)
        for cid in se.ranking_dedupe:
            print('Structure ID {0:>6} keeps as the elite'.format(cid))
            elite_struc[cid] = opt_struc_data[cid]
            elite_fitness[cid] = all_fitness[cid]
    else:
        elite_struc = None
        elite_fitness = None
    # ---------- out
    with open('cryspy.out', 'a') as fout:
        fout.write('{} structures keeps as the elite\n'.format(rin.n_elite))

    # ---------- new generation
    gen += 1

    # ---------- id_queueing
    id_queueing = [i for i in range(rin.tot_struc, rin.tot_struc + rin.n_pop)]

    # ---------- ea_info
    tmp_info = pd.Series([gen, rin.n_pop, rin.n_crsov, rin.n_perm,
                          rin.n_strain, rin.n_rand, rin.n_elite,
                          rin.crs_lat, rin.slct_func],
                         index=ea_info.columns)
    ea_info = ea_info.append(tmp_info, ignore_index=True)
    # ------ out ea_info
    out_results.out_ea_info(ea_info)

    # ---------- ea_origin
    # ------ EA operation part
    for cid in range(rin.tot_struc, rin.tot_struc + rin.n_pop - rin.n_rand):
        tmp_origin = pd.Series([gen, cid, eagen.operation[cid],
                                eagen.parents[cid]], index=ea_origin.columns)
        ea_origin = ea_origin.append(tmp_origin, ignore_index=True)
    # ------ random part
    for cid in range(rin.tot_struc + rin.n_pop - rin.n_rand,
                     rin.tot_struc + rin.n_pop):
        tmp_origin = pd.Series([gen, cid, 'random', None],
                               index=ea_origin.columns)
        ea_origin = ea_origin.append(tmp_origin, ignore_index=True)
    # ------ elite part
    if rin.n_elite > 0:
        for cid in se.ranking_dedupe:
            tmp_origin = pd.Series([gen, cid, 'elite', 'elite'],
                                   index=ea_origin.columns)
            ea_origin = ea_origin.append(tmp_origin, ignore_index=True)
    # ------ out ea_origin

    _ea_origin_dict = convert_df_to_aiida_type(ea_origin)
    out_results.out_ea_origin(ea_origin)

    # ---------- save ea_id_data
    ea_id_data = (gen, id_queueing, id_running)
    _ea_id_data_dict = {'gen': gen, 'id_queueing':id_queueing, 'id_running': id_running }
    pkl_data.save_ea_id(ea_id_data)

    # ---------- save ea_data
    ea_data = (elite_struc, elite_fitness, ea_info, ea_origin)
    _ea_data_dict = convert_struc_dict_to_aiidatype(elite_struc)
    _elite_fitness_dict = convert_struc_dict_to_aiidatype(elite_fitness)
    _ea_info_dict = convert_df_to_aiida_type(ea_info)
    _ea_origin_dict = convert_df_to_aiida_type(ea_origin)
    pkl_data.save_ea_data(ea_data)

    # ---------- change the value of tot_struc
    config = change_input.config_read()
    change_input.change_basic(config, 'tot_struc', rin.tot_struc + rin.n_pop)
    change_input.write_config(config)
    print('# -- changed cryspy.in')
    print('Changed the value of tot_struc in cryspy.in'
          ' from {} to {}'.format(
              rin.tot_struc, rin.tot_struc + rin.n_pop))

    # ---------- status
    io_stat.set_input_common(stat, 'basic', 'tot_struc', rin.tot_struc + rin.n_pop)
    io_stat.set_common(stat, 'generation', gen)
    io_stat.set_id(stat, 'id_queueing', id_queueing)
    _stat = stat._sections
    io_stat.write_stat(stat)


def next_gen_EA(job):
    # ---------- log and out
    with open('cryspy.out', 'a') as fout:
        fout.write('\nDone generation {}\n\n'.format(job.gen))
    print('\nDone generation {}\n'.format(job.gen))
    # ---------- check point 3
    if rin.stop_chkpt == 3:
        print('\nStop at check point 3: EA is ready\n')
        # os.remove('lock_cryspy')
        raise SystemExit()
    # ---------- maxgen_ea
    if 0 < rin.maxgen_ea <= job.gen:
        print('\nReached maxgen_ea: {}\n'.format(rin.maxgen_ea))
        # os.remove('lock_cryspy')
        raise SystemExit()
    # ---------- EA
    ea_id_data = (job.gen, job.id_queueing, job.id_running)
    
    stat_dict =jobs.stat._sections
    init_struc_dic = convert_struc_dict_to_aiidatype(jobs.init_struc_data)
    opt_struc_dic = convert_struc_dict_to_aiidatype(jobs.opt_struc_data)
    rslt_data_dic = convert_df_to_aiida_type(jobs.rslt_data)
    ea_id_data_dic = {'gen':job.gen, 'id_queueing': job.id_queueing, 
                      'id_running': job.id_running}

    next_gen_dict(stat_dict, init_struc_dic,
                         opt_struc_dic, rslt_data_dic, ea_id_data_dic)
    

def next_sg(job):
    '''
    next selection or generation
    '''
    if rin.algo == 'BO':
        # job.next_select_BO()
        raise ValueError('not supported.')
    if rin.algo == 'LAQA':
        # job.next_select_LAQA()
        raise ValueError('not supported.')
    if rin.algo == 'EA':
        next_gen_EA(job)


In [ ]:
print(len(jobs.id_queueing), len(jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    next_sg(jobs)
    

# next run

In [ ]:
init_struc_data.keys()

In [ ]:
stat, struc_dict = cryspy_restart.restart()
struc_dict

In [ ]:
jobs = Ctrl_job(stat, struc_dict) # init_struc_dataの更新の必要がある。
jobs.check_job()

In [ ]:
stat["basic"]["tot_struc"]

In [ ]:
cwd_dict, struc_dict = handle_job_return_dict(jobs)

In [ ]:
jobs.init_struc_data

In [ ]:
jobs.id_queueing, jobs.id_running


In [ ]:
cwd_dict, struc_dict


In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)
    

In [ ]:
print(len(jobs.id_queueing), len(jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    jobs.next_sg()
    

# next run


In [ ]:
stat, struc_dict = cryspy_restart.restart()
jobs = Ctrl_job(stat, struc_dict)
jobs.check_job()
cwd_dict, struc_dict = handle_job_return_dict(jobs)


In [ ]:
jobs.id_queueing, jobs.id_running

In [ ]:
cwd_dict, struc_dict

In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)

In [ ]:
print(len(jobs.id_queueing), len(jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    jobs.next_sg()

In [ ]:
!verdi graph generate 14891

In [ ]:
!pwd